# Домашняя работа "Классификация в АОТ"

## Классификация текстов
### Fakenews
1. Мы будем работать с данными fakenews отсюда: https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv   
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации.
3. Векторизуйте.
4. Обучите на полученных векторах алгоритм классификации.   
Мы уже видели как эта задача выполняется с помощью Word2vec. Давайте вспомним.

3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn и выше 0.52 для методов на pytorch.

In [1]:
# Загружаем библиотеки
import pandas as pd
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import nltk
nltk.download('punkt')
from gensim.models.word2vec import Word2Vec
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Загружаем данные
df = pd.read_csv('Constraint_Train.csv')
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [3]:
# Получаем предложения/твиты
sentences = [word_tokenize(text.lower()) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:03<00:00, 2002.30it/s]


In [4]:
# Обучаем модель Word2Vec
%time model_tweets = Word2Vec(sentences, workers=4, size=300, min_count=3, window=5, iter=15)

CPU times: user 12.5 s, sys: 83.6 ms, total: 12.5 s
Wall time: 11.2 s


In [5]:
# Смотрим какие слова оказались близки к слову france
model_tweets.wv.most_similar('france')

[('2015', 0.9422296285629272),
 ('front', 0.9320982098579407),
 ('bags', 0.929604172706604),
 ('jamaat', 0.9159358143806458),
 ('victims', 0.9144309163093567),
 ('invented', 0.9141560792922974),
 ('parliament', 0.9136128425598145),
 ('2017', 0.9129412770271301),
 ('protesting', 0.9119986891746521),
 ('saudi', 0.9107229113578796)]

In [6]:
# Нормируем вектора
model_tweets.init_sims()

In [7]:
# Функция создаёт эмбединг текста
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.sum(result, axis=0)
    else:
        result = np.zeros(300)
    return result

In [8]:
# Делаем вектора признаков
features = [get_text_embedding(text) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:08<00:00, 718.27it/s]


In [9]:
# Разделяем данные на train и test 
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)


### Способ 1 - логистическая регрессия

In [10]:
# Обучаем модель
model_1 = LogisticRegression(solver='liblinear', random_state=0)
model_1.fit(X_train, y_train)

LogisticRegression(random_state=0, solver='liblinear')

In [11]:
# Делаем предсказание
predicted_1 = model_1.predict(X_test)

In [12]:
# Смотрим точность модели
f1_score(y_test, predicted_1, average=None)

array([0.91891892, 0.92243767])

### Способ 2 - метод опорных векторов

In [13]:
# Обучаем модель
model_2 = svm.SVC(kernel='linear', gamma='scale', random_state=0)
model_2.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

In [14]:
# Делаем предсказание
predicted_2 = model_2.predict(X_test)

In [15]:
# Смотрим точность модели
f1_score(y_test, predicted_2, average=None)

array([0.91800097, 0.92237023])

### Способ 3 - k ближайших соседей 

In [16]:
# Обучаем модель
model_3 = KNeighborsClassifier(n_neighbors=3)
model_3.fit(X_train, y_train) 

KNeighborsClassifier(n_neighbors=3)

In [17]:
# Делаем предсказание
predicted_3 = model_3.predict(X_test)

In [18]:
# Смотрим точность модели
f1_score(y_test, predicted_3, average=None)

array([0.91936263, 0.92293493])